In [1]:
from open3d import *
import numpy as np
from scipy.io import loadmat

import torch
import torch.utils.data as data

import os
from os import listdir
from os.path import isfile, join

In [2]:
def load_directory_names(filename):
    
    directory_list = []
    
    file = open(filename, 'r') 
    lines = file.readlines() 

    for line in lines: 
        line_string = line.strip()
        split_string = line_string.split(" ", 2)
        
        directory_list.append(split_string[1])
        
    return directory_list

In [3]:
# get all filenames in a given folder 

def get_all_filenames_from_directory(directory_path):
    onlyfiles = [f for f in listdir(directory_path) if isfile(join(directory_path, f))]
    
    onlyfiles.sort()
    
    return onlyfiles

In [4]:
def seperate_file_list_into_pcd_and_label(file_list):
    
    file_list.sort()
    
    pcd_list = [];
    label_list = []
    
    suffix_pcd = ".pcd"
    suffix_mat = ".mat"
    
    for file in file_list:
        
        if(file.endswith(suffix_pcd)):
            pcd_list.append(file)
            
        if(file.endswith(suffix_mat)):
            label_list.append(file)
            
    
    pcd_list.sort()
    label_list.sort()
    
    return pcd_list, label_list

In [5]:
class Dataset(data.Dataset):
    
    def __init__(self, dataset_root, dataset_folders, number_of_points, category):
        
        self.dataset_root = dataset_root
        # list of folders in root folder
        self.dataset_folders = load_directory_names(dataset_folders)
        self.category = category
        self.number_of_points = number_of_points
        
        self.pointcloud_list = []
        self.label_list = []
        
        # load labels and pointclouds into self.pointcloud_list and self.label_list
        self.load_point_clouds()
        
    
    def load_point_clouds(self):
        
        # iterate over all folders
        for folder in self.dataset_folders:
            
            # if first part of folder name equals category name
            if(folder.split('_')[0] == self.category):
                
                # get all filenames in given folder
                file_list = get_all_filenames_from_directory(self.dataset_root + '/' + folder)
                
                pcd_files, label_files = seperate_file_list_into_pcd_and_label(file_list)
                
                # iterate over all files in folder, already ensured that pcd_files and label_files have same length
                for i in range (0, len(pcd_files)):
                    
                    # load pointcloud
                    pcd = open3d.io.read_point_cloud(self.dataset_root + '/' + folder + '/' + pcd_files[i])
                    # load labels
                    labels = loadmat(self.dataset_root + '/' + folder + '/' + label_files[i])
                    
                    self.pointcloud_list.append(pcd)
                    self.label_list.append(labels)  
                    
                    
    def __getitem__(self, index):
        
        labels = self.label_list[index]['data'][0]
        points = np.array(self.pointcloud_list[index].points)
        
        # randomly choose number of points form point cloud and labels 
        choice = np.random.choice(len(points), self.number_of_points, replace=True)
        
        points = points[choice, :]
        labels = labels[choice]
        labels = np.array(labels) 
        
        points = points.astype(np.float32)
        
        torch_labels = torch.from_numpy(labels)
        torch_points = torch.from_numpy(points)
        return torch_points, torch_labels 
    
    
    def __len__(self):
        return len(self.pointcloud_list)
    